[23 Tablas dinámicas](#23)

# 22 Group By: extraer valor con variables categóricas

In [1]:
import pandas as pd
import numpy as np
pd.__version__

'1.3.1'

In [2]:
pd.options.display.float_format = '{:,.2f}'.format


In [3]:
import seaborn as sns

In [4]:
df = sns.load_dataset('tips')
df

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4
...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,Dinner,3
240,27.18,2.00,Female,Yes,Sat,Dinner,2
241,22.67,2.00,Male,Yes,Sat,Dinner,2
242,17.82,1.75,Male,No,Sat,Dinner,2


In [5]:
df['prct_tip'] = df['tip']/df['total_bill']

In [6]:
df['ones'] = 1

In [9]:
df.head(5)

,total_bill,tip,sex,smoker,day,time,size,prct_tip,ones
0,16.99,1.01,Female,No,Sun,Dinner,2,0.06,1
1,10.34,1.66,Male,No,Sun,Dinner,3,0.16,1
2,21.01,3.50,Male,No,Sun,Dinner,3,0.17,1
3,23.68,3.31,Male,No,Sun,Dinner,2,0.14,1
4,24.59,3.61,Female,No,Sun,Dinner,4,0.15,1


conteo del total de peronas que componen esos grupos:


In [10]:
df_g = df.groupby(['sex','smoker'])[['ones']].sum()
df_g

ones
sex    smoker      
Male   Yes       60
       No        97
Female Yes       33
       No        54

agrupando en el nivel del género

In [11]:
# sacando el porcentaje de fumadores por género
# a cada grupo le aplica la función 
# para los hombre, x toma el valor de 60 y 97
df_g.groupby(level=0).apply(

lambda x:
x / x.sum() * 100

)

ones
sex    smoker      
Male   Yes    38.22
       No     61.78
Female Yes    37.93
       No     62.07

El 38% de hombres fuma y el 61 no.

Otra forma de obtener valor de nuestros datos, podemos transformar nuestras variables numéricas a una variable categórica, a pesar de ser continua, por ello usamos `cut` para crear las categorías por bins, osea por intervalos.

In [12]:
pd.cut(df['total_bill'],bins = 3)

0       (3.022, 18.983]
1       (3.022, 18.983]
2      (18.983, 34.897]
3      (18.983, 34.897]
4      (18.983, 34.897]
             ...       
239    (18.983, 34.897]
240    (18.983, 34.897]
241    (18.983, 34.897]
242     (3.022, 18.983]
243     (3.022, 18.983]
Name: total_bill, Length: 244, dtype: category
Categories (3, interval[float64, right]): [(3.022, 18.983] < (18.983, 34.897] < (34.897, 50.81]]

In [13]:
pd.cut(df['total_bill'],bins = 3).value_counts()

(3.022, 18.983]     140
(18.983, 34.897]     88
(34.897, 50.81]      16
Name: total_bill, dtype: int64

Podemos definir el ancho de nuestras categorías, especificando `bins ` como una lista:

In [15]:
pd.cut(df['total_bill'], bins = [3,18,35,60]).value_counts()

(3, 18]     127
(18, 35]    101
(35, 60]     16
Name: total_bill, dtype: int64

podemos crear una columna nueva con éstso datos categóricos:

In [ ]:
df['bin_total'] = pd.cut(df['total_bill'],bins = [3,18,35,60])

In [17]:
df.head(5)

,total_bill,tip,sex,smoker,day,time,size,prct_tip,ones,bin_total
0,16.99,1.01,Female,No,Sun,Dinner,2,0.06,1,"(3, 18]"
1,10.34,1.66,Male,No,Sun,Dinner,3,0.16,1,"(3, 18]"
2,21.01,3.50,Male,No,Sun,Dinner,3,0.17,1,"(18, 35]"
3,23.68,3.31,Male,No,Sun,Dinner,2,0.14,1,"(18, 35]"
4,24.59,3.61,Female,No,Sun,Dinner,4,0.15,1,"(18, 35]"


In [18]:
df.dtypes

total_bill     float64
tip            float64
sex           category
smoker        category
day           category
time          category
size             int64
prct_tip       float64
ones             int64
bin_total     category
dtype: object

ésta nueva categoría podemos agruparla. veamos la relación del monto con el tipo de comida (comida o cena). y hagamos el porcentaje por valor de la cuenta, agrupado por tipo de comida. Usamos la misma función lambda que usamos anteriormente.

In [19]:
df.groupby(['time','bin_total'])[['ones']].count()

ones
time   bin_total      
Lunch  (3, 18]      45
       (18, 35]     21
       (35, 60]      2
Dinner (3, 18]      82
       (18, 35]     80
       (35, 60]     14

In [20]:
df.groupby(['time','bin_total'])[['ones']].count().groupby(level=0).apply(

lambda x:
x / x.sum() * 100

)

ones
time   bin_total      
Lunch  (3, 18]   66.18
       (18, 35]  30.88
       (35, 60]   2.94
Dinner (3, 18]   46.59
       (18, 35]  45.45
       (35, 60]   7.95

Vemos que menos del 3 % de las cuentas superan los 35 dolares, durante la comida, y que durante la cena, el monto aumenta.

# 23 Tablas dinámicas , Pivot table <a class="anchor" id="23"></a>

Para mostrar el uso veamos primero el resultado de obtener **la media** por grupos y generar un dataframe con el resultado

In [ ]:
df.groupby(['sex','time'])[['total_bill']].mean()

total_bill
sex    time              
Male   Lunch       18.048
       Dinner      21.461
Female Lunch       16.339
       Dinner      19.213

Creamos un nuevo dataflame con índices

In [21]:
df_gp = df.groupby(['sex','time'])[['total_bill']].mean().reset_index()
df_gp

,sex,time,total_bill
0,Male,Lunch,18.05
1,Male,Dinner,21.46
2,Female,Lunch,16.34
3,Female,Dinner,19.21


Vemos que tenemos dos grupos, género y tiempo, y un valor respectivo  para cada dato.

Una mejor forma de vizualizarlo es pasando el segundo grupo como columnas, para ello pusamos `.pivot_table`, con los siguientes parámetros:
- `values` los valores que se mostrarán dentro de la tabla
- `index` el nombre de las filas
- `columns` el nombre de las columnas

In [22]:
df_gp.pivot_table(values='total_bill', index='sex', columns='time')

time,Lunch,Dinner
sex,,
Male,18.05,21.46
Female,16.34,19.21


Ahora nuestros grupos coupan los lugares de indice y columna. además de que los valores se muestran en el respectivo cruce

<img src="https://pandas-docs.github.io/pandas-docs-travis/_images/reshaping_pivot.png" alt="drawing" width="600"/>

podríamos haber aplicado el `.pivot table` directamente al dataframe sin necesidad de crear el dataframe agrupado, vemos que el resultado es el mismo. Esto es por que por defaul pivot table obtiene la media de los valores.

In [23]:
df.pivot_table(values='total_bill', index='sex', columns='time')

time,Lunch,Dinner
sex,,
Male,18.05,21.46
Female,16.34,19.21


Por default `.pivot_table` obtiene la media, pero podríamos definir qué función aplicar `aggfunc`

In [26]:
df.pivot_table(values='total_bill', index='sex', columns='time', aggfunc='median')
# o podemos usar np.median sin comillas

time,Lunch,Dinner
sex,,
Male,16.58,19.63
Female,13.42,17.19


In [27]:
df_pivot = df.pivot_table(values='total_bill', index='sex', columns='time', aggfunc=[np.median,'std'])
df_pivot

median          std       
time    Lunch Dinner Lunch Dinner
sex                              
Male    16.58  19.63  7.95   9.46
Female  13.42  17.19  7.50   8.20

Parara deshacer las categorías usamos `.unstack()` , lo cual devuelve al parecer una serie con múltiples indices

In [34]:
df_pivot.unstack()

        time    sex   
median  Lunch   Male     16.58
                Female   13.42
        Dinner  Male     19.63
                Female   17.19
std     Lunch   Male      7.95
                Female    7.50
        Dinner  Male      9.46
                Female    8.20
dtype: float64

In [35]:
type(df_pivot.unstack())

pandas.core.series.Series

podemos convertir la serie a un dataframe (que también tendrá multiples índices)

In [36]:
df_pivot.unstack().reset_index()

,level_0,time,sex,0
0,median,Lunch,Male,16.58
1,median,Lunch,Female,13.42
2,median,Dinner,Male,19.63
3,median,Dinner,Female,17.19
4,std,Lunch,Male,7.95
5,std,Lunch,Female,7.50
6,std,Dinner,Male,9.46
7,std,Dinner,Female,8.20
